# Análisis de Víctimas en carpetas de investigación FGJ

Objetivo:
* Implementar un modelo del clasificación para el análsis de las víctimas en carpetas de investigación FGJ en la Ciudad de México
* Datos: https://datos.cdmx.gob.mx/dataset/victimas-en-carpetas-de-investigacion-fgj

## Bibliotecas y setup

In [ ]:
# Dependencies
import pandas as pd 
pd.set_option('display.max_columns', 100)
import numpy as np 
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#
import warnings 
warnings.filterwarnings('ignore') # Para ignorar warnings

### Nota: Si alguna biblioteca no se instala utilizar el siguiente código

In [ ]:
# 
#!pip install pip # Cuando utilicen por primera vez pip
#!pip install <<name_library>>

## SECCIÓN I:

### ¿Qué nos dicen los datos? 

Se trata de datos que contienen la información actualizada de las víctimas de los delitos en las carpetas de investigación de la Fiscalía General de Justicia (FGJ) de la Ciudad de México a partir de enero de 2019.

Para una correcta interpretación de la información, la CDMX hizó las siguientes aclaraciones:

* El campo "fecha_hecho" representa la fecha en la que ocurrió el hecho.
* El campo "fecha_inicio" corresponde a la fecha de la apertura de la carpeta de investigación.
* En esta base se señala el sexo de la víctima, así como la fecha en que ocurrieron los hechos denunciados.
* Es importante destacar que, aunque en algunas ocasiones la víctima es la persona denunciante, en otras, denunciante y víctima son personas diferentes (por ejemplo, casos en los que menores de edad son víctimas).
* Es posible que en una misma denuncia se incluya a una o más víctimas.
* Existen otras aclaraciones en: https://datos.cdmx.gob.mx/dataset/victimas-en-carpetas-de-investigacion-fgj

**Los datos de esta tabla fueron actualizados por la FGJ el 29 de julio de 2020.**

Los datos antes de la actualización del 29 de julio de 2020 los puedes consultar en esta liga: Victimas antes de la actualización de julio de 2020 [https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/victimas-en-carpetas-de-investigacion-fgj/victimas_ss_junio2020.csv]

In [ ]:
# Consulta a una dirección Web que aloja un archivo CSV (DATOS ACTUALIZADOS)

Datos = pd.read_csv('https://archivo.datos.cdmx.gob.mx/FGJ/victimas/victimasFGJ_acumulado_2024_09.csv')

Datos.tail()

In [ ]:
#

Datos.shape

In [ ]:
#

Datos.info()

In [ ]:
# Consulta a una dirección Web que aloja un archivo CSV (DATOS NO ACTUALIZADOS)

Datos_OLD = pd.read_csv('https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/victimas-en-carpetas-de-investigacion-fgj/victimas_ss_junio2020.csv')

Datos_OLD.tail()

In [ ]:
#

Datos_OLD.shape

In [ ]:
#

Datos_OLD.info()

In [ ]:
# Agrupamos por año-mes y categoria_delito de los DATOS ACTUALIZADOS

Datos['Periodo'] = Datos['fecha_hecho'].str[:7]

Time_Serie = Datos[ ['Periodo', 'categoria_delito', 
                     'anio_inicio'] ].groupby(by = ['Periodo', 
                                                    'categoria_delito']).count().reset_index( drop = False)

Time_Serie = Time_Serie[ Time_Serie['Periodo'] >= '2019-01' \
                       ].reset_index( drop = True ).rename( columns = {'categoria_delito': 'Categoria', 
                                                                       'anio_inicio' : 'Numero'} )

Time_Serie

In [ ]:
#

Time_Serie = pd.pivot_table( Time_Serie, values = 'Numero', index = ['Periodo'], 
                             columns = ['Categoria'], aggfunc = 'sum' ).reset_index( drop = False )

Time_Serie

In [ ]:
# Agrupamos por año-mes y categoria_delito de los DATOS NO ACTUALIZADOS

Datos_OLD['Periodo'] = Datos_OLD['FechaHecho'].str[6:10] + '-' + Datos_OLD['FechaHecho'].str[3:5]

Time_Serie_OLD = Datos_OLD[ ['Periodo', 'Categoria', 
                             'idCarpeta'] ].groupby(by = ['Periodo', 
                                                          'Categoria']).count().reset_index( drop = False)

Time_Serie_OLD = Time_Serie_OLD[ Time_Serie_OLD['Periodo'] >= '2019-01'\
                               ].reset_index( drop = True ).rename( columns = {'idCarpeta' : 'Numero'} )

Time_Serie_OLD

In [ ]:
#

Time_Serie_OLD = pd.pivot_table( Time_Serie_OLD, values = 'Numero', index = ['Periodo'], 
                                 columns = ['Categoria'], aggfunc = 'sum' ).reset_index( drop = False )

Time_Serie_OLD

In [ ]:
#

Time_Serie = Time_Serie.drop('HECHO NO DELICTIVO', axis=1)
Time_Serie = Time_Serie.drop('SECUESTRO', axis=1)
Time_Serie = Time_Serie.drop('HOMICIDIO DOLOSO', axis=1)
Time_Serie = Time_Serie.drop('VIOLACIÓN', axis=1)
Time_Serie = Time_Serie.drop('LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', axis=1)
Time_Serie = Time_Serie.drop('ROBO A CASA HABITACIÓN CON VIOLENCIA', axis=1)
Time_Serie = Time_Serie.drop('ROBO A NEGOCIO CON VIOLENCIA', axis=1)
Time_Serie = Time_Serie.drop('ROBO A REPARTIDOR CON Y SIN VIOLENCIA', axis=1)

In [ ]:
#
plt.style.available

In [ ]:
# Visualización

plt.style.use('seaborn-v0_8-darkgrid') # 'ggplot', 'seaborn-v0_8-darkgrid'

plt.figure( figsize = ( 15, 6 ) )

colors = cm.rainbow(np.linspace(0, 1, len(Time_Serie.columns) - 1))
 
# Plot multiple lines
for num, column in enumerate(Time_Serie.drop('Periodo', axis=1)):
    plt.plot(Time_Serie['Periodo'], Time_Serie[column], 
             marker='', color=colors[num], linewidth=1, 
             alpha=0.9, label=column)

# Add legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
 
# Add titles
plt.title( 'Evolución de Victimas por Categorías de Crimen en CDMX (DATOS AJUSTADOS)', 
           loc = 'left', fontsize = 11, fontweight = 0, color = 'darkblue' )
plt.xlabel( 'Periodo' )
plt.ylabel( '' )

plt.xticks( rotation = 90 )

# Adjust layout to make room for the legend
plt.tight_layout()

In [ ]:
#

Time_Serie = Time_Serie.drop('DELITO DE BAJO IMPACTO', axis=1)

In [ ]:
# Visualización

plt.style.use('ggplot') # 'ggplot', 'seaborn-v0_8-darkgrid'

plt.figure( figsize = ( 15, 6 ) )

colors = cm.rainbow(np.linspace(0, 1, len(Time_Serie.columns) - 1))
 
# Plot multiple lines
for num, column in enumerate(Time_Serie.drop('Periodo', axis=1)):
    plt.plot(Time_Serie['Periodo'], Time_Serie[column], 
             marker='', color=colors[num], linewidth=1, 
             alpha=0.9, label=column)

# Add legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
 
# Add titles
plt.title( 'Evolución de Victimas por Categorías de Crimen en CDMX (DATOS AJUSTADOS)', 
           loc = 'left', fontsize = 11, fontweight = 0, color = 'darkblue' )
plt.xlabel( 'Periodo' )
plt.ylabel( '' )

plt.xticks( rotation = 90 )

# Adjust layout to make room for the legend
plt.tight_layout()

In [ ]:
#

Time_Serie_OLD = Time_Serie_OLD.drop('HECHO NO DELICTIVO', axis=1)
Time_Serie_OLD = Time_Serie_OLD.drop('SECUESTRO', axis=1)
Time_Serie_OLD = Time_Serie_OLD.drop('HOMICIDIO DOLOSO', axis=1)
Time_Serie_OLD = Time_Serie_OLD.drop('VIOLACIÓN', axis=1)
Time_Serie_OLD = Time_Serie_OLD.drop('LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', axis=1)
Time_Serie_OLD = Time_Serie_OLD.drop('ROBO A CASA HABITACIÓN CON VIOLENCIA', axis=1)
Time_Serie_OLD = Time_Serie_OLD.drop('ROBO A NEGOCIO CON VIOLENCIA', axis=1)
Time_Serie_OLD = Time_Serie_OLD.drop('ROBO A REPARTIDOR CON Y SIN VIOLENCIA', axis=1)

In [ ]:
# Visualización

plt.style.use('ggplot') # 'ggplot', 'seaborn-darkgrid'

plt.figure( figsize = ( 15, 6 ) )

colors = cm.rainbow(np.linspace(0, 1, len(Time_Serie_OLD.columns) - 1))
 
# Plot multiple lines
for num, column in enumerate(Time_Serie_OLD.drop('Periodo', axis=1)):
    plt.plot(Time_Serie_OLD['Periodo'], Time_Serie_OLD[column], 
             marker='', color=colors[num], linewidth=1, 
             alpha=0.9, label=column)

# Add legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
 
# Add titles
plt.title( 'Evolución de Victimas por Categorías de Crimen en CDMX (DATOS NO AJUSTADOS)', 
           loc = 'left', fontsize = 11, fontweight = 0, color = 'darkblue' )
plt.xlabel( 'Periodo' )
plt.ylabel( '' )

plt.xticks( rotation = 90 )

# Adjust layout to make room for the legend
plt.tight_layout()

In [ ]:
#

Time_Serie_OLD = Time_Serie_OLD.drop('DELITO DE BAJO IMPACTO', axis=1)

In [ ]:
# Visualización

plt.style.use('ggplot') # 'ggplot', 'seaborn-darkgrid'

plt.figure( figsize = ( 15, 6 ) )

colors = cm.rainbow(np.linspace(0, 1, len(Time_Serie_OLD.columns) - 1))
 
# Plot multiple lines
for num, column in enumerate(Time_Serie_OLD.drop('Periodo', axis=1)):
    plt.plot(Time_Serie_OLD['Periodo'], Time_Serie_OLD[column], 
             marker='', color=colors[num], linewidth=1, 
             alpha=0.9, label=column)

# Add legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
 
# Add titles
plt.title( 'Evolución de Victimas por Categorías de Crimen en CDMX (DATOS NO AJUSTADOS)', 
           loc = 'left', fontsize = 11, fontweight = 0, color = 'darkblue' )
plt.xlabel( 'Periodo' )
plt.ylabel( '' )

plt.xticks( rotation = 90 )

# Adjust layout to make room for the legend
plt.tight_layout()

## SECCIÓN II:

### Limpieza de datos

In [ ]:
# Selección de información para el modelo
# Usaremos los DATOS ACTUALIZADOS:

Datos_ML = Datos[ Datos['Periodo'] >= '2019-01' ]

Datos_ML = Datos_ML[ [ 'Periodo','fecha_inicio', 'hora_inicio', 'fecha_hecho', 'hora_hecho', 'categoria_delito',
                       'sexo', 'edad', 'tipo_persona', 'calidad_juridica', 'latitud', 'longitud' ] ]

Datos_ML.info()

In [ ]:
#

Datos_ML.Periodo.value_counts()

In [ ]:
#

Datos_ML.categoria_delito.value_counts()

In [ ]:
#

Datos_ML.tipo_persona.value_counts()

In [ ]:
#

pd.crosstab( Datos_ML['Periodo'], Datos_ML['tipo_persona'] , normalize = 'index' )

In [ ]:
#

Datos_ML = Datos_ML[ Datos_ML['tipo_persona'] != 'MORAL' ]

In [ ]:
#

Datos_ML.calidad_juridica.value_counts()

In [ ]:
#
Datos_ML['personas'] = 'NA'
#
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'VICTIMA Y DENUNCIANTE', 'personas'] = 'Victima_y_Otra'
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'OFENDIDO Y DENUNCIANTE', 'personas'] = 'Victima_y_Otra'
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'DENUNCIANTE Y VICTIMA', 'personas'] = 'Victima_y_Otra'
#
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'VICTIMA', 'personas'] = 'Victima'
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'OFENDIDO', 'personas'] = 'Victima'
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'AGRAVIADO', 'personas'] = 'Victima'
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'LESIONADO', 'personas'] = 'Victima'
Datos_ML.loc[ Datos_ML['calidad_juridica'] == 'CADAVER', 'personas'] = 'Victima'

#Datos_ML.personas.value_counts()
Datos_ML = Datos_ML[ Datos_ML['personas'] != 'NA' ]

In [ ]:
#

Datos_ML.personas.value_counts()

In [ ]:
# Creamos una variable de días

Datos_ML['Dias'] = ( pd.to_timedelta( 
                     pd.to_datetime( Datos_ML['fecha_inicio'] + '-' + Datos_ML['hora_inicio'],
                                     format = '%Y-%m-%d-%H:%M:%S' ) - \
                     pd.to_datetime( Datos_ML['fecha_hecho'] + '-' + Datos_ML['hora_hecho'], 
                                     format = '%Y-%m-%d-%H:%M:%S' ) 
                   ).dt.total_seconds() / 3600 ) / 24

In [ ]:
# Un histograma 

Datos_ML['Dias'].hist( bins = 30 )

plt.show()

In [ ]:
# Filtramos casos en los que no transcurrió más de 1 año entre la denuncia y el evento:

Datos_ML = Datos_ML[ ( Datos_ML['Dias'] >= 0 ) & ( Datos_ML['Dias'] <= 365 ) ]

Datos_ML['Dias'].hist( bins = 30 )

plt.show()

In [ ]:
#
# Extraer el día de la semana 

Datos_ML['fecha_hecho_dia'] = pd.to_datetime( Datos_ML['fecha_hecho'] ).dt.day_name()

colores_dias = {
    'Monday': 'red',
    'Tuesday': 'orange',
    'Wednesday': 'yellow',
    'Thursday': 'green',
    'Friday': 'blue',
    'Saturday': 'purple',
    'Sunday': 'pink'
}

# Crear el gráfico
plt.figure(figsize=(10,6))
orden_dias = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

barras = sn.countplot(
    x="fecha_hecho_dia", 
    data=Datos_ML,
    order=orden_dias,  # Para que salgan en orden correcto
    palette=[colores_dias[dia] for dia in orden_dias]
)

# Mostrar
plt.title('Cantidad de hechos por día de la semana')
plt.xlabel('Día de la semana')
plt.ylabel('Cantidad')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#
# Extraer hora

Datos_ML['hora_hecho_hora'] = pd.to_datetime( Datos_ML['hora_hecho'] ).dt.hour

Datos_ML['hora_hecho_hora'].hist( bins = 24 )

plt.show()

In [ ]:
#

Datos_ML.sexo.value_counts()

In [ ]:
#

Datos_ML['edad'].hist( bins = 30 )

plt.show()

In [ ]:
# Filtramos casos en los que no transcurrió más de 1 año entre la denuncia y el evento:

Datos_ML = Datos_ML[ ( Datos_ML['edad'] >= 0 ) & ( Datos_ML['edad'] <= 100 ) ]

Datos_ML['edad'].hist( bins = 30 )

plt.show()

In [ ]:
# 

Datos_ML.info()

In [ ]:
# Filtramos casos en los que no hay datos de longitud y latitud:

#Datos_ML[ ( Datos_ML['latitud'].isnull() ) | ( Datos_ML['longitud'].isnull() ) ]

Datos_ML = Datos_ML[ ( Datos_ML['latitud'].notnull() ) | ( Datos_ML['longitud'].notnull() ) ]

Datos_ML.info()

In [ ]:
#

Datos_ML.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 0.1)

plt.show()

In [ ]:
#

Datos_ML = Datos_ML[ [ 'Periodo', 'categoria_delito', 'fecha_inicio', 'hora_inicio',
                       'fecha_hecho', 'hora_hecho', 'fecha_hecho_dia', 'hora_hecho_hora',
                       'sexo', 'edad', 'personas', 'Dias', 'latitud', 'longitud' ] ]

In [ ]:
#

Datos.shape, Datos_ML.shape, 100 * Datos_ML.shape[0] / Datos.shape[0]

## SECCIÓN III:

### Selección de datos

* Usemos los datos previos a Julio de 2020 para entrenar un modelo que permita clasificar "ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA"

In [ ]:
# Agrupamos por año-mes y categoria_delito de los DATOS ACTUALIZADOS

Time_Serie_ML = Datos_ML[ ['Periodo', 'categoria_delito', 
                           'fecha_inicio'] ].groupby(
                                 by = ['Periodo','categoria_delito']
                           ).count().reset_index( drop = False).rename( 
                                   columns = {'categoria_delito': 'Categoria', 
                                              'fecha_inicio' : 'Numero'} )

Time_Serie_ML

In [ ]:
#

Time_Serie_ML = pd.pivot_table( Time_Serie_ML, values = 'Numero', index = ['Periodo'], 
                                columns = ['Categoria'], aggfunc = 'sum' ).reset_index( drop = False )

Time_Serie_ML

In [ ]:
#
Time_Serie_ML = Time_Serie_ML.drop('HECHO NO DELICTIVO', axis=1)
Time_Serie_ML = Time_Serie_ML.drop('SECUESTRO', axis=1)
Time_Serie_ML = Time_Serie_ML.drop('HOMICIDIO DOLOSO', axis=1)
Time_Serie_ML = Time_Serie_ML.drop('VIOLACIÓN', axis=1)
Time_Serie_ML = Time_Serie_ML.drop('LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', axis=1)
Time_Serie_ML = Time_Serie_ML.drop('ROBO A CASA HABITACIÓN CON VIOLENCIA', axis=1)
Time_Serie_ML = Time_Serie_ML.drop('ROBO A NEGOCIO CON VIOLENCIA', axis=1)
Time_Serie_ML = Time_Serie_ML.drop('ROBO A REPARTIDOR CON Y SIN VIOLENCIA', axis=1)

In [ ]:
# Visualización

plt.style.use('ggplot') # 'ggplot', 'seaborn-darkgrid'

plt.figure( figsize = ( 15, 6 ) )

colors = cm.rainbow(np.linspace(0, 1, len(Time_Serie_ML.columns) - 1))
 
# Plot multiple lines
for num, column in enumerate(Time_Serie_ML.drop('Periodo', axis=1)):
    plt.plot(Time_Serie_ML['Periodo'], Time_Serie_ML[column], 
             marker='', color=colors[num], linewidth=1, 
             alpha=0.9, label=column)

# Add legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
 
# Add titles
plt.title( 'Evolución de Victimas por Categorías de Crimen en CDMX (DATOS AJUSTADOS Y FILTRADOS)', 
           loc = 'left', fontsize = 11, fontweight = 0, color = 'darkblue' )
plt.xlabel( 'Periodo' )
plt.ylabel( '' )

plt.xticks( rotation = 90 )

# Adjust layout to make room for the legend
plt.tight_layout()

In [ ]:
#

Time_Serie_ML = Time_Serie_ML.drop('DELITO DE BAJO IMPACTO', axis=1)

In [ ]:
# Visualización

plt.style.use('ggplot') # 'ggplot', 'seaborn-darkgrid'

plt.figure( figsize = ( 15, 6 ) )

colors = cm.rainbow(np.linspace(0, 1, len(Time_Serie_ML.columns) - 1))
 
# Plot multiple lines
for num, column in enumerate(Time_Serie_ML.drop('Periodo', axis=1)):
    plt.plot(Time_Serie_ML['Periodo'], Time_Serie_ML[column], 
             marker='', color=colors[num], linewidth=1, 
             alpha=0.9, label=column)

# Add legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
 
# Add titles
plt.title( 'Evolución de Victimas por Categorías de Crimen en CDMX (DATOS AJUSTADOS)', 
           loc = 'left', fontsize = 11, fontweight = 0, color = 'darkblue' )
plt.xlabel( 'Periodo' )
plt.ylabel( '' )

plt.xticks( rotation = 90 )

# Adjust layout to make room for the legend
plt.tight_layout()

In [ ]:
# Selección de información para el modelo

# Usaremos los DATOS ACTUALIZADOS de 2019 como conjunto para entrenar y probar nuestro modelo:

Datos_01 = Datos_ML[ ( Datos_ML['Periodo'] >= '2019-01' ) & \
                     ( Datos_ML['Periodo'] <= '2019-12' ) ].reset_index( drop = True )

# Usaremos los DATOS ACTUALIZADOS de 2023 como conjunto para reclasificar delitos con nuestro modelo:

Datos_02 = Datos_ML[ ( Datos_ML['Periodo'] >= '2023-01' ) & \
                     ( Datos_ML['Periodo'] <= '2023-12' ) ].reset_index( drop = True )

#
Datos_01.shape, Datos_02.shape, 100*Datos_01.shape[0]/Datos_ML.shape[0], 100*Datos_02.shape[0]/Datos_ML.shape[0]

In [ ]:
#

Datos_01 = pd.concat( [ Datos_01, pd.get_dummies( Datos_01['sexo'], dtype = float) ], axis = 1 )

Datos_01 = pd.concat( [ Datos_01, pd.get_dummies( Datos_01['personas'], dtype = float) ], axis = 1 )

Datos_01 = pd.concat( [ Datos_01, pd.get_dummies( Datos_01['fecha_hecho_dia'].astype(str), dtype = float) ], axis = 1 )

Datos_01 = pd.concat( [ Datos_01, pd.get_dummies( Datos_01['hora_hecho_hora'].astype(str), dtype = float) ], axis = 1 )

Datos_01 = Datos_01[ [ 'Periodo', 'categoria_delito', 'fecha_hecho', 'hora_hecho', 
                       'fecha_hecho_dia', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
                       'Friday', 'Saturday', 'Sunday', 'hora_hecho_hora', '0', '1', '2', 
                       '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 
                       '15', '16', '17', '18', '19', '20', '21', '22', '23', 'Dias', 
                       'sexo', 'Femenino', 'Masculino', 'edad', 'personas', 'Victima', 
                       'Victima_y_Otra', 'latitud', 'longitud',] ]

In [ ]:
#

Datos_01.info()

In [ ]:
#

Datos_02 = pd.concat( [ Datos_02, pd.get_dummies( Datos_02['sexo'], dtype = float) ], axis = 1 )

Datos_02 = pd.concat( [ Datos_02, pd.get_dummies( Datos_02['personas'], dtype = float) ], axis = 1 )

Datos_02 = pd.concat( [ Datos_02, pd.get_dummies( Datos_02['fecha_hecho_dia'].astype(str), dtype = float) ], axis = 1 )

Datos_02 = pd.concat( [ Datos_02, pd.get_dummies( Datos_02['hora_hecho_hora'].astype(str), dtype = float) ], axis = 1 )

Datos_02 = Datos_02[ [ 'Periodo', 'categoria_delito', 'fecha_hecho', 'hora_hecho', 
                       'fecha_hecho_dia', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
                       'Friday', 'Saturday', 'Sunday', 'hora_hecho_hora', '0', '1', '2', 
                       '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 
                       '15', '16', '17', '18', '19', '20', '21', '22', '23', 'Dias', 
                       'sexo', 'Femenino', 'Masculino', 'edad', 'personas', 'Victima', 
                       'Victima_y_Otra', 'latitud', 'longitud',] ]

In [ ]:
#

Datos_02.info()

In [ ]:
#

Datos_01.describe()

In [ ]:
#

Datos_02.describe()

## SECCIÓN IV:

### Modelado

* Usemos los datos previos a Julio de 2020 para entrenar un modelo que permita clasificar "ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA"

* Plantearemos un modelo de respuesta binaria:

Este tipo de modelos suponene que existe una variable latente que se puede expresar como una ecuación lineal dada por:
$$y^*_i = \mathbf{x}_i \boldsymbol{\beta} + \varepsilon_i$$

Donde $\varepsilon_i$ es una variable aleatoria con función de densidad con media cero y distribución simetrica al rededor de cero. Dado lo anterior, para nosotros sólo es visible que:
\begin{equation*}
    y_i = 
    \begin{cases}
        1 & \text{si } y^*_i > 0 \\
        0 & \text{si } y^*_i < 0 
    \end{cases}
\end{equation*}

Visto en nuestro caso, podemos pensar que estamos ante algo como:
\begin{equation*}
    y_i = 
    \begin{cases}
        \text{ ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA } & \text{si } y^*_i > 0 \\
        \text{ OTRO CASO } & \text{si } y^*_i < 0 
    \end{cases}
\end{equation*}

De esta forma tenemos una estrucutura de la probabilidad dada por:
\begin{eqnarray*}
    P(y_i = 1 | \mathbf{x}_i) & = & P(\mathbf{x}_i \boldsymbol{\beta} + \varepsilon_i > 0) = P(\varepsilon_i > - \mathbf{x}_i \boldsymbol{\beta}) = G(\mathbf{x}_i \boldsymbol{\beta}) \\
    P(y_i = 0 | \mathbf{x}_i) & = & P(\mathbf{x}_i \boldsymbol{\beta} + \varepsilon_i < 0) = P(\varepsilon_i < - \mathbf{x}_i \boldsymbol{\beta}) = 1 - G(\mathbf{x}_i \boldsymbol{\beta})
\end{eqnarray*}

Donde $\mathbf{x}_i$ es un vector de dimensión $K \times 1$ que contiene al menos el término constante y $\boldsymbol{\beta}$ es un vector de parámetros a estimar, de forma que asumiremos:
\begin{equation*}
    \mathbf{x}_i \boldsymbol{\beta} = \beta_1 + x_{i2} \beta_2 + \ldots + x_{iK} \beta_K
\end{equation*}

Asumiremos que $G(\cdot)$ es uan función de densidad acumulada de forma que:
\begin{equation*}
    0< G(\mathbf{x}_i \boldsymbol{\beta}) < 1 \text{ , } \forall \mathbf{x}_i \boldsymbol{\beta} \in \mathbb{R}
\end{equation*}

En este caso utilizaremos dos modelos que dependen de la forma funcional de $G(\cdot)$ que está determinada por la distribución de $\varepsilon_i$. De esta forma tendremos dos modelos: Probit y Logit.

In [ ]:
#
Datos_01.shape

In [ ]:
# Seleccionamos solo los casos de interés para Dataset 01

Datos_01 = Datos_01[ ( Datos_01['categoria_delito'] == 'DELITO DE BAJO IMPACTO' ) | \
                     ( Datos_01['categoria_delito'] == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA' ) ]

# Variable categórica:
Datos_01[ 'Delito' ] = 0

Datos_01.loc[ Datos_01['categoria_delito'] == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA', 'Delito'] = 1

In [ ]:
#
Datos_01.shape

In [ ]:
#

sn.countplot( x = 'Delito', data = Datos_01)

plt.show()

In [ ]:
#
Datos_01.Delito.describe()

In [ ]:
#
Datos_02.shape

In [ ]:
# Seleccionamos solo los casos de interés para Dataset 02

Datos_02 = Datos_02[ ( Datos_02['categoria_delito'] == 'DELITO DE BAJO IMPACTO' ) | \
                     ( Datos_02['categoria_delito'] == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA' ) ]

# Variable categórica:
Datos_02[ 'Delito' ] = 0

Datos_02.loc[ Datos_02['categoria_delito'] == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA', 'Delito'] = 1

In [ ]:
#
Datos_02.shape

In [ ]:
#

sn.countplot( x = 'Delito', data = Datos_02)

plt.show()

In [ ]:
#
Datos_02.Delito.describe()

### Logistic Regression
Logistic regression is a type of regression analysis in statistics used for prediction of outcome of a categorical dependent variable from a set of predictor or independent variables. In logistic regression the dependent variable is always binary. Logistic regression is mainly used to for prediction and also calculating the probability of success.

In [ ]:
# Add constant

Datos_01[ 'Constante' ] = 1 

In [ ]:
# Selecciones las columnas de nuestra regresión:

Datos_01.columns

In [ ]:
# Model

model = sm.Logit( 
    Datos_01.Delito, # Y
    Datos_01[[ 'Constante', 
               'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', # Quitamos el domingo
               '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 
               '15', '16', '17', '18', '19', '20', '21', '22', '23', # Quitamos la hora 0
               'Femenino', # Quitamos la dummy de masculino
               'Victima', # Quitamos la dummy de cuando la denuncia la hace alguien más que la victima
               'Dias', 'edad', 'latitud', 'longitud']]
)

result = model.fit()

result.summary()

In [ ]:
#

print(result.summary())

In [ ]:
# Rebalanceamos la muestra:

# 1. Selección de muestra 32% del caso delito 0
Datos_01_train, Datos_01_test = train_test_split( Datos_01[ Datos_01['Delito'] == 0 ], 
                                                  test_size = 0.68, random_state = 42 )

# 2. Juntamos delitos 0 y 1
Datos_01_train = pd.concat( [ Datos_01[ Datos_01['Delito'] == 1 ], Datos_01_train ], ignore_index = True )

Datos_01.shape, Datos_01_train.shape

In [ ]:
#

sn.countplot( x = 'Delito', data = Datos_01_train)

plt.show()

In [ ]:
#
Datos_01_train.Delito.describe()

In [ ]:
#
Datos_01_train.describe()

In [ ]:
#
Datos_01.describe()

In [ ]:
# Model

model_adj = sm.Logit( 
    Datos_01_train.Delito,
    Datos_01_train[[ 'Constante', 
                     'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', # Quitamos el domingo
                     '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 
                     '15', '16', '17', '18', '19', '20', '21', '22', '23', # Quitamos la hora 0
                     'Femenino', # Quitamos la dummy de masculino
                     'Victima', # Quitamos la dummy de cuando la denuncia la hace alguien más que la victima
                     'Dias', 'edad', 'latitud', 'longitud']]
)

result_adj = model_adj.fit()
print(result_adj.summary())

### Implementemos / entrenemos nuestro modelo

In [ ]:
#

X = Datos_01_train[[ 'Constante', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 
                     '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 
                     '15', '16', '17', '18', '19', '20', '21', '22', '23', 
                     'Femenino', 'Victima', 'Dias', 'edad', 'latitud', 'longitud']]
y = Datos_01_train.Delito

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.20, random_state = 5)

In [ ]:
# Del sklearn

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

In [ ]:
# Model accuracy

accuracy_score( y_test, y_pred )

In [ ]:
# Confusion matrix

cm = confusion_matrix( y_test, y_pred )

conf_matrix = pd.DataFrame( data = cm, 
                            columns = ['Predicted:0','Predicted:1'],
                            index = ['Actual:0','Actual:1'] )

conf_matrix

In [ ]:
#

plt.figure( figsize = (8,5) )

sn.heatmap( conf_matrix, annot = True, fmt = 'd', cmap = "YlGnBu" )

plt.show()

In [ ]:
# accuracy

(7316 + 430) / (7316 + 430 + 344 + 2180)

In [ ]:
# Model Evaluation - Statistics

TN=cm[0,0]
TP=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
sensitivity=TP/float(TP+FN)
specificity=TN/float(TN+FP)

print('The acuuracy of the model = TP+TN/(TP+TN+FP+FN) = ',(TP+TN)/float(TP+TN+FP+FN),'\n',

'The Missclassification = 1-Accuracy = ',1-((TP+TN)/float(TP+TN+FP+FN)),'\n',

'Sensitivity or True Positive Rate = TP/(TP+FN) = ',TP/float(TP+FN),'\n',

'Specificity or True Negative Rate = TN/(TN+FP) = ',TN/float(TN+FP),'\n',

'Positive Predictive value = TP/(TP+FP) = ',TP/float(TP+FP),'\n',

'Negative predictive Value = TN/(TN+FN) = ',TN/float(TN+FN),'\n',

'Positive Likelihood Ratio = Sensitivity/(1-Specificity) = ',sensitivity/(1-specificity),'\n',

'Negative likelihood Ratio = (1-Sensitivity)/Specificity = ',(1-sensitivity)/specificity)

In [ ]:
# Calculamos las probabiliades:

y_pred_prob    = logreg.predict_proba( X_test )[: , :]

y_pred_prob_df = pd.DataFrame( data = y_pred_prob, 
                               columns = ['Prob de delito bajo impacto (0)','Prob de robo (1)'] )

y_pred_prob_df.head()

### ROC curve

Una forma común de visualizar las compensaciones de diferentes umbrales es mediante el uso de una curva ROC, un gráfico de la tasa de verdaderos positivos (# verdaderos positivos/# total de positivos) versus la tasa de falsos positivos (# falsos positivos/# total de negativos) para todas las opciones posibles de umbrales. Un modelo con buena precisión de clasificación debería tener significativamente más positivos verdaderos que falsos positivos en todos los umbrales.

La posición óptima para la curva roc es hacia la esquina superior izquierda, donde la especificidad y la sensibilidad están en niveles óptimos.

Área bajo la curva (AUC). El área bajo la curva ROC cuantifica la precisión de la clasificación del modelo; cuanto mayor sea el área, mayor será la disparidad entre los verdaderos y falsos positivos, y más fuerte será el modelo para clasificar los miembros del conjunto de datos de entrenamiento. Un área de 0,5 corresponde a un modelo que no funciona mejor que la clasificación aleatoria y un buen clasificador se mantiene lo más lejos posible de eso. Un área de 1 es ideal. Cuanto más cerca esté el AUC de 1, mejor.

In [ ]:
#

fpr, tpr, thresholds = roc_curve( y_test, y_pred_prob_df.iloc[:,1] )

#
plt.plot(fpr,tpr)
plt.xlim([0.0, 1.01])
plt.ylim([0.0, 1.01])
plt.title('ROC curve for theft crime')
plt.xlabel('False positive rate (1-Specificity)')
plt.ylabel('True positive rate (Sensitivity)')
plt.grid(True)

In [ ]:
#

roc_auc_score(y_test, y_pred_prob_df.iloc[:,1])

## SECCIÓN V:

### Prediciendo sobre el dataset que queremos clasificar

* Usemos los datos de 2023 para clasificar "ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA"

In [ ]:
# Seleccionamos solo los casos de interés para Dataset 02

Datos_02 = Datos_02[ ( Datos_02['categoria_delito'] == 'DELITO DE BAJO IMPACTO' ) | \
                     ( Datos_02['categoria_delito'] == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA' ) ]

# Variable categórica:
Datos_02[ 'Delito' ] = 0

Datos_02.loc[ Datos_02['categoria_delito'] == 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA', 'Delito'] = 1

In [ ]:
#

sn.countplot( x = 'Delito', data = Datos_02)

plt.show()

In [ ]:
#

Datos_02.describe()

In [ ]:
#

Datos_02[ 'Constante' ] = 1

X_New = Datos_02[[ 'Constante', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 
                   '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', 
                   '15', '16', '17', '18', '19', '20', '21', '22', '23', 
                   'Femenino', 'Victima', 'Dias', 'edad', 'latitud', 'longitud']]

Datos_02[ 'Delito_Predict' ] = logreg.predict(X_New)

In [ ]:
#

sn.countplot( x = 'Delito_Predict', data = Datos_02)

plt.show()

In [ ]:
#

Datos_02[ ['Delito', 'Delito_Predict'] ].describe()